In [1]:
import random
import networkx as nx
import pandas as pd
f = pd.read_csv("higgs-activity_time.txt.gz",sep=" ",names=['source','target','timestamp','type'])
d = pd.read_csv("higgs-social_network.edgelist.gz",sep=" ",names=['source','target'])

g = nx.from_pandas_edgelist(f[:10000],source="source",target="target",edge_attr = ["timestamp","type"])

In [72]:
g_l = list(g.edges(data=True))
g_l = sorted(g_l,key = lambda x: x[2]['timestamp'])

In [73]:
#g_neigh = pd.DataFrame(d.groupby('source')['target'].apply(list))

In [74]:
g_n = d.groupby('source').count().to_dict(orient='dict')['target']
for i in g_n.keys():
    g_n[i] = [g_n[i],"inactive",0,0,False,0,set(),-1]

In [75]:
g_n[1]

[22, 'inactive', 0, 0, False, 0, set(), -1]

In [76]:
graph = {'links' : g_l, 'nodes' : g_n}

In [77]:
def LT(graph):
    active = []
    blocked = []
    count = 0
    for i in graph['links']:
        for j in blocked:
            if graph['nodes'][j][5] - 1 == 0:
                graph['nodes'][j][4] = False
                graph['nodes'][j][5] = 0
                blocked.remove(j)
            else:
                graph['nodes'][j][5] -= 1
        
        for j in active:
            if graph['nodes'][j][2] - 1 == 0:
                graph['nodes'][j][2] = 0
                graph['nodes'][j][1] = "inactive"
                active.remove(j)
            else:
                graph['nodes'][j][2] -= 1
                if i[2]['timestamp'] - graph['nodes'][j][3] > 10000:
                    #print(f"Rumor propogation minimized at {i[2]['timestamp']} by node {j}")
                    graph['nodes'][j][2] = 0
                    graph['nodes'][j][1] = "inactive"
                    active.remove(j)
                    r_node = graph['nodes'][j][7]
                    if graph['nodes'][r_node][4] == False:
                        temp_count = 0
                        for temp in graph['nodes'][r_node][6]:
                            if (graph['nodes'][temp][1] == "inactive" or (i[2]['timestamp'] - graph['nodes'][temp][3] > 5000)) and graph['nodes'][temp][2] > 0:
                                temp_count += 1
                        if temp_count > 0.4*len(graph['nodes'][r_node][6]):
                            graph['nodes'][r_node][4] = True
                            graph['nodes'][r_node][5] = 200
                            blocked.append(r_node)
                            print(f"Rumor spreading node {r_node} blocked at {i[2]['timestamp']}")
                            count += 1
                    else:
                        pass
                else:
                    pass
            
        graph['nodes'][i[0]][6].add(i[1])
        
        if graph['nodes'][i[0]][4] == True:
            pass
        else:
            if graph['nodes'][i[1]][1] == "inactive":
                if 1/graph['nodes'][i[0]][0] > 1/graph['nodes'][i[1]][0]:
                    graph['nodes'][i[1]][1] = "active"
                    graph['nodes'][i[1]][2] = 200
                    graph['nodes'][i[1]][3] = i[2]['timestamp']
                    graph['nodes'][i[1]][7] = i[0]
                    active.append(i[1])
            else:
                pass
            
    print(count)

In [78]:
LT(graph)

Rumor spreading node 196145 blocked at 1341115951
Rumor spreading node 9192 blocked at 1341119598
Rumor spreading node 281951 blocked at 1341121634
Rumor spreading node 186184 blocked at 1341121844
Rumor spreading node 88 blocked at 1341129395
Rumor spreading node 12510 blocked at 1341131597
Rumor spreading node 222663 blocked at 1341132360
Rumor spreading node 2156 blocked at 1341140228
Rumor spreading node 29233 blocked at 1341144799
Rumor spreading node 87449 blocked at 1341149489
10
